In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic
import pandas as pd
import nltk
from nltk.corpus import stopwords
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/Masterarbeit/BertClassifierEnv/News/news_categorised_final_all_Mig_NER.xlsx")
data

,link,text,title,lead,tags,source,language,is_alt_news,word_count,migration,tags_migration,identical,opinion,Entities
0,https://abendblatt.de/hamburg/kommunales/artic...,Hamburg. Es gab Zeiten in der seit 2015 besteh...,Rot-grüne Gereiztheiten und Sticheleien im Rat...,Die Sozialdemokraten werfen dem Koalitionspart...,Hamburger Senat,abendblatt,de,0,1082,1,0,0,0,"['LOC: Hamburg', 'ORG: SPD', 'ORG: Grünen', 'P..."
1,https://abendblatt.de/hamburg/polizeimeldungen...,Cuxhaven/Hamburg. Kurz nach dem Start einer Ai...,Seenotretter holen krankes Kleinkind von „Aida...,Kurz nach dem Start der Kreuzfahrt in Hamburg ...,Kreuzfahrt,abendblatt,de,0,120,1,0,0,0,"['LOC: Cuxhaven', 'LOC: Hamburg', 'LOC: Hambur..."
2,https://abendblatt.de/politik/article233076529...,"Berlin. Es sind Bilder, die einen nicht so sch...","""Menschliche Überreste"" an Flugzeug gefunden –...",Verzweifelte klammerten sich in Kabul an ein U...,Afghanistan,abendblatt,de,0,371,1,0,0,0,"['LOC: Berlin', 'LOC: Kabul', 'ORG: Taliban', ..."
3,https://abendblatt.de/politik/article233452685...,"Berlin/Kiel. Daniel Günther tut, wovon andere ...",Günther zu Jamaika: „Jede Partei muss Kröten s...,CDU-Ministerpräsident Daniel Günther über sein...,Interview,abendblatt,de,0,544,1,0,0,0,"['LOC: Berlin', 'LOC: Kiel', 'PER: Daniel Günt..."
4,https://abendblatt.de/region/schleswig-holstei...,Helgoland. Ein Unfall auf dem Kreuzfahrtschiff...,Unfall auf der „Europa“ – Frau vor Helgoland i...,Seenotretter übernehmen auf der Nordsee eine v...,Kreuzfahrt,abendblatt,de,0,196,1,0,0,0,"['LOC: Helgoland', 'MISC: Europa', 'LOC: Helgo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4809,https://zeit.de/zett/politik/2021-07/oez-atten...,"Da ist Can, vielleicht drei oder vier Jahre al...",Gegen die Stille,Vor fünf Jahren starb Can Leyla bei einem Ansc...,"München, Utøya, Hanau, Rassismus, Kriminalität...",zeit,de,0,862,1,0,0,0,"['PER: Can', 'PER: Can', 'PER: Can', 'PER: Has..."
4810,https://zeit.de/zett/politik/2021-08/shoan-vai...,"Eine geflüchtete Person, die in Deutschland an...",Er wäre der erste Geflüchtete im Parlament,Vor zehn Jahren floh der Kurde Shoan Vaisi aus...,"Bundestagswahl, Die Linke, Bundestag, Geflücht...",zeit,de,0,576,1,0,0,0,"['LOC: Deutschland', 'PER: Shoan Vaisi', 'LOC:..."
4811,https://zeit.de/zett/politik/2021-09/diversita...,Auch der neue Bundestag wird die Gesellschaft ...,Diese Menschen machen den Bundestag diverser,Zum ersten Mal sitzen zwei trans Politikerinne...,"Bundestag, Bundestagswahl, Diskriminierung, Gl...",zeit,de,0,991,1,0,0,0,"['PER: Nyke Slawik', 'PER: Tessa Ganserer', 'O..."
4812,https://zeit.de/zett/politik/2021-09/wahlrecht...,Zu seiner ersten Wahl trägt Avni* kurze Joggin...,Seine erste Wahl,Avni darf zum ersten Mal den Bundestag wählen:...,"Wahlrecht, Bundestagswahl, Bundesrepublik Deut...",zeit,de,0,865,1,0,0,0,"['PER: Avni', 'LOC: Berlin', 'PER: Avni', 'PER..."


In [ ]:
# delete all duplicate articles
data = data.drop_duplicates(subset='title', keep='first')

In [ ]:
len(data)

3096

In [ ]:
# remove WELT+ articles containing pseudonymized words
data = data[~data['text'].str.contains('uuu')]
len(data)

2977

In [ ]:
# remove Morgenpost+ articles containing pseudonymized words
data = data[~data['text'].str.contains('ejf')]
len(data)

2966

In [ ]:
#remove all articles from rp-online (not readable paid articles 36)
# Entferne alle Zeilen, in denen 'rp_online' in der Spalte 'source' steht
data = data[data['source'] != 'rp_online']

len(data)

2932

In [ ]:
# for reproducible results (https://maartengr.github.io/BERTopic/faq.html#why-are-the-results-not-consistent-between-runs)
umap_model = UMAP(random_state=1)

# 1 good

# before removing WELT+ random_state = 123

In [ ]:
# remove stop words (https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html)
nltk.download('stopwords')
german_stop_words = stopwords.words('german')

vectorizer_model = CountVectorizer(stop_words = german_stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
umap_model = UMAP(random_state=12)

In [ ]:
# trying to further reduce topics (https://maartengr.github.io/BERTopic/getting_started/topicreduction/topicreduction.html)
# execute 2nd topic model
topic_model2 = BERTopic(language = "multilingual", vectorizer_model=vectorizer_model, calculate_probabilities=True, umap_model=umap_model, nr_topics = "auto")
topics2, probs2 = topic_model2.fit_transform(list(data["text"]))

In [ ]:

# look at first results of 2nd topic model
topic_model2.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,980,-1_mehr_menschen_deutschland_sei,"[mehr, menschen, deutschland, sei, wurde, scho...","[Schlauchboote vor der libyschen Küste, Flücht..."
1,0,563,0_taliban_afghanistan_kabul_menschen,"[taliban, afghanistan, kabul, menschen, sagte,...",[In der usbekischen Hauptstadt Taschkent ist i...
2,1,446,1_eu_belarus_grenze_lukaschenko,"[eu, belarus, grenze, lukaschenko, migranten, ...","[Update vom Dienstag, 16.11.2021, 08.20 Uhr: D..."
3,2,333,2_spd_prozent_partei_grünen,"[spd, prozent, partei, grünen, laschet, cdu, a...","[Herr Klingbeil, Anfang Mai haben Sie uns zu e..."
4,3,101,3_polizei_mann_wurde_jährige,"[polizei, mann, wurde, jährige, sei, worden, z...",[Drei Wochen nach dem mörderischen Messerangri...
5,4,85,4_stadt_sagt_greifswald_wohnungen,"[stadt, sagt, greifswald, wohnungen, wurde, me...","[Ringsum Bäume und Wiesen, alles in sattem Grü..."
6,5,55,5_merkel_kanzlerin_angela_merkels,"[merkel, kanzlerin, angela, merkels, 16, polit...","[Finanzkrise 2008, Flüchtlingskrise 2015, Covi..."
7,6,43,6_rio_haiti_del_mexiko,"[rio, haiti, del, mexiko, migranten, us, usa, ...",[Im Süden der USA spielt sich eine akute Migra...
8,7,41,7_sea_mittelmeer_migranten_bord,"[sea, mittelmeer, migranten, bord, watch, mens...",[Das deutsche Rettungsschiff Sea-Watch 3 darf ...
9,8,33,8_polizei_feuerwehr_feuer_flammen,"[polizei, feuerwehr, feuer, flammen, brände, m...",[Am 18. Oktober 2018 gegen 2.20 Uhr ging im sä...


In [ ]:
# Reduce outliers for 2nd topic model (using probabilities) -> works fine (https://maartengr.github.io/BERTopic/getting_started/outlier_reduction/outlier_reduction.html)
new_topics2 = topic_model2.reduce_outliers(list(data["text"]), topics2, probabilities=probs2, strategy="probabilities")
topic_model2.update_topics(list(data["text"]), topics=new_topics2, vectorizer_model=vectorizer_model)

2023-11-27 13:51:31,920 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [ ]:
topic_model2.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,654,0_taliban_afghanistan_kabul_menschen,"[taliban, afghanistan, kabul, menschen, sagte,...",[In der usbekischen Hauptstadt Taschkent ist i...
1,1,613,1_eu_belarus_grenze_migranten,"[eu, belarus, grenze, migranten, polen, lukasc...","[Update vom Dienstag, 16.11.2021, 08.20 Uhr: D..."
2,2,802,2_grünen_spd_mehr_cdu,"[grünen, spd, mehr, cdu, deutschland, prozent,...","[Herr Klingbeil, Anfang Mai haben Sie uns zu e..."
3,3,306,3_wurde_polizei_mann_menschen,"[wurde, polizei, mann, menschen, jährige, sagt...",[Drei Wochen nach dem mörderischen Messerangri...
4,4,85,4_stadt_sagt_greifswald_wohnungen,"[stadt, sagt, greifswald, wohnungen, wurde, me...","[Ringsum Bäume und Wiesen, alles in sattem Grü..."
5,5,55,5_merkel_kanzlerin_angela_merkels,"[merkel, kanzlerin, angela, merkels, 16, polit...","[Finanzkrise 2008, Flüchtlingskrise 2015, Covi..."
6,6,44,6_rio_haiti_migranten_del,"[rio, haiti, migranten, del, mexiko, us, usa, ...",[Im Süden der USA spielt sich eine akute Migra...
7,7,46,7_sea_migranten_mittelmeer_menschen,"[sea, migranten, mittelmeer, menschen, bord, w...",[Das deutsche Rettungsschiff Sea-Watch 3 darf ...
8,8,34,8_feuerwehr_polizei_flammen_feuer,"[feuerwehr, polizei, flammen, feuer, brände, w...",[Am 18. Oktober 2018 gegen 2.20 Uhr ging im sä...
9,9,31,9_türkei_erdogan_türkische_türkischen,"[türkei, erdogan, türkische, türkischen, türke...",[Beim EU-Türkei-Gipfel in der bulgarischen Sch...


In [ ]:
topic_model2.get_topics()

{0: [('taliban', 0.03453377424599084),
  ('afghanistan', 0.03452994316854672),
  ('kabul', 0.024329407963660565),
  ('menschen', 0.019053142078305732),
  ('sagte', 0.015852875231454056),
  ('deutschland', 0.015597186549011618),
  ('ortskräfte', 0.014985371544668993),
  ('us', 0.014586083629325612),
  ('afghanen', 0.01449160510091525),
  ('flughafen', 0.014379247930976436)],
 1: [('eu', 0.03246964616893439),
  ('belarus', 0.025812860386399156),
  ('grenze', 0.025713065396693238),
  ('migranten', 0.023020835265627967),
  ('polen', 0.021606260269869587),
  ('lukaschenko', 0.021183838391153162),
  ('menschen', 0.01786306461333778),
  ('flüchtlinge', 0.013927108704062522),
  ('deutschland', 0.013903327707910447),
  ('sagte', 0.012406417367214057)],
 2: [('grünen', 0.011838697665826783),
  ('spd', 0.01181858790088396),
  ('mehr', 0.011344130639778174),
  ('cdu', 0.010723084367126226),
  ('deutschland', 0.010320668527205175),
  ('prozent', 0.010306635414813227),
  ('afd', 0.01027505284098201)

In [ ]:
topic_model2.visualize_documents(list(data["text"]), hide_document_hover=True, hide_annotations=True)

In [ ]:
topic_model2.visualize_topics()

In [ ]:
# merge results of first topic model to origin dataset
data["Topic"] = new_topics2
data

,link,text,title,lead,tags,source,language,is_alt_news,word_count,migration,tags_migration,identical,opinion,Entities,Topic
0,https://abendblatt.de/hamburg/kommunales/artic...,Hamburg. Es gab Zeiten in der seit 2015 besteh...,Rot-grüne Gereiztheiten und Sticheleien im Rat...,Die Sozialdemokraten werfen dem Koalitionspart...,Hamburger Senat,abendblatt,de,0,1082,1,0,0,0,"['LOC: Hamburg', 'ORG: SPD', 'ORG: Grünen', 'P...",2
1,https://abendblatt.de/hamburg/polizeimeldungen...,Cuxhaven/Hamburg. Kurz nach dem Start einer Ai...,Seenotretter holen krankes Kleinkind von „Aida...,Kurz nach dem Start der Kreuzfahrt in Hamburg ...,Kreuzfahrt,abendblatt,de,0,120,1,0,0,0,"['LOC: Cuxhaven', 'LOC: Hamburg', 'LOC: Hambur...",1
2,https://abendblatt.de/politik/article233076529...,"Berlin. Es sind Bilder, die einen nicht so sch...","""Menschliche Überreste"" an Flugzeug gefunden –...",Verzweifelte klammerten sich in Kabul an ein U...,Afghanistan,abendblatt,de,0,371,1,0,0,0,"['LOC: Berlin', 'LOC: Kabul', 'ORG: Taliban', ...",0
3,https://abendblatt.de/politik/article233452685...,"Berlin/Kiel. Daniel Günther tut, wovon andere ...",Günther zu Jamaika: „Jede Partei muss Kröten s...,CDU-Ministerpräsident Daniel Günther über sein...,Interview,abendblatt,de,0,544,1,0,0,0,"['LOC: Berlin', 'LOC: Kiel', 'PER: Daniel Günt...",2
4,https://abendblatt.de/region/schleswig-holstei...,Helgoland. Ein Unfall auf dem Kreuzfahrtschiff...,Unfall auf der „Europa“ – Frau vor Helgoland i...,Seenotretter übernehmen auf der Nordsee eine v...,Kreuzfahrt,abendblatt,de,0,196,1,0,0,0,"['LOC: Helgoland', 'MISC: Europa', 'LOC: Helgo...",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4805,https://zeit.de/zeit-magazin/leben/2021-08/tan...,"Empathie verbindet Menschen, sie ermöglicht un...","""Empathie ist nicht zu verwechseln mit Mitgefühl""","Krieg, Flut, Brände: Aus der Ferne empfinden w...",NaN,zeit,de,0,104,1,0,0,0,"['LOC: Afghanistan', 'LOC: Haiti', 'PER: Tania...",0
4809,https://zeit.de/zett/politik/2021-07/oez-atten...,"Da ist Can, vielleicht drei oder vier Jahre al...",Gegen die Stille,Vor fünf Jahren starb Can Leyla bei einem Ansc...,"München, Utøya, Hanau, Rassismus, Kriminalität...",zeit,de,0,862,1,0,0,0,"['PER: Can', 'PER: Can', 'PER: Can', 'PER: Has...",3
4810,https://zeit.de/zett/politik/2021-08/shoan-vai...,"Eine geflüchtete Person, die in Deutschland an...",Er wäre der erste Geflüchtete im Parlament,Vor zehn Jahren floh der Kurde Shoan Vaisi aus...,"Bundestagswahl, Die Linke, Bundestag, Geflücht...",zeit,de,0,576,1,0,0,0,"['LOC: Deutschland', 'PER: Shoan Vaisi', 'LOC:...",1
4811,https://zeit.de/zett/politik/2021-09/diversita...,Auch der neue Bundestag wird die Gesellschaft ...,Diese Menschen machen den Bundestag diverser,Zum ersten Mal sitzen zwei trans Politikerinne...,"Bundestag, Bundestagswahl, Diskriminierung, Gl...",zeit,de,0,991,1,0,0,0,"['PER: Nyke Slawik', 'PER: Tessa Ganserer', 'O...",2


In [ ]:
merged_data = pd.merge(data, topic_model2.get_topic_info(), on="Topic", how="left")

In [ ]:
merged_data

,link,text,title,lead,tags,source,language,is_alt_news,word_count,migration,tags_migration,identical,opinion,Entities,Topic,Count,Name,Representation,Representative_Docs
0,https://abendblatt.de/hamburg/kommunales/artic...,Hamburg. Es gab Zeiten in der seit 2015 besteh...,Rot-grüne Gereiztheiten und Sticheleien im Rat...,Die Sozialdemokraten werfen dem Koalitionspart...,Hamburger Senat,abendblatt,de,0,1082,1,0,0,0,"['LOC: Hamburg', 'ORG: SPD', 'ORG: Grünen', 'P...",2,802,2_grünen_spd_mehr_cdu,"[grünen, spd, mehr, cdu, deutschland, prozent,...","[Herr Klingbeil, Anfang Mai haben Sie uns zu e..."
1,https://abendblatt.de/hamburg/polizeimeldungen...,Cuxhaven/Hamburg. Kurz nach dem Start einer Ai...,Seenotretter holen krankes Kleinkind von „Aida...,Kurz nach dem Start der Kreuzfahrt in Hamburg ...,Kreuzfahrt,abendblatt,de,0,120,1,0,0,0,"['LOC: Cuxhaven', 'LOC: Hamburg', 'LOC: Hambur...",1,613,1_eu_belarus_grenze_migranten,"[eu, belarus, grenze, migranten, polen, lukasc...","[Update vom Dienstag, 16.11.2021, 08.20 Uhr: D..."
2,https://abendblatt.de/politik/article233076529...,"Berlin. Es sind Bilder, die einen nicht so sch...","""Menschliche Überreste"" an Flugzeug gefunden –...",Verzweifelte klammerten sich in Kabul an ein U...,Afghanistan,abendblatt,de,0,371,1,0,0,0,"['LOC: Berlin', 'LOC: Kabul', 'ORG: Taliban', ...",0,654,0_taliban_afghanistan_kabul_menschen,"[taliban, afghanistan, kabul, menschen, sagte,...",[In der usbekischen Hauptstadt Taschkent ist i...
3,https://abendblatt.de/politik/article233452685...,"Berlin/Kiel. Daniel Günther tut, wovon andere ...",Günther zu Jamaika: „Jede Partei muss Kröten s...,CDU-Ministerpräsident Daniel Günther über sein...,Interview,abendblatt,de,0,544,1,0,0,0,"['LOC: Berlin', 'LOC: Kiel', 'PER: Daniel Günt...",2,802,2_grünen_spd_mehr_cdu,"[grünen, spd, mehr, cdu, deutschland, prozent,...","[Herr Klingbeil, Anfang Mai haben Sie uns zu e..."
4,https://abendblatt.de/region/schleswig-holstei...,Helgoland. Ein Unfall auf dem Kreuzfahrtschiff...,Unfall auf der „Europa“ – Frau vor Helgoland i...,Seenotretter übernehmen auf der Nordsee eine v...,Kreuzfahrt,abendblatt,de,0,196,1,0,0,0,"['LOC: Helgoland', 'MISC: Europa', 'LOC: Helgo...",7,46,7_sea_migranten_mittelmeer_menschen,"[sea, migranten, mittelmeer, menschen, bord, w...",[Das deutsche Rettungsschiff Sea-Watch 3 darf ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2927,https://zeit.de/zeit-magazin/leben/2021-08/tan...,"Empathie verbindet Menschen, sie ermöglicht un...","""Empathie ist nicht zu verwechseln mit Mitgefühl""","Krieg, Flut, Brände: Aus der Ferne empfinden w...",NaN,zeit,de,0,104,1,0,0,0,"['LOC: Afghanistan', 'LOC: Haiti', 'PER: Tania...",0,654,0_taliban_afghanistan_kabul_menschen,"[taliban, afghanistan, kabul, menschen, sagte,...",[In der usbekischen Hauptstadt Taschkent ist i...
2928,https://zeit.de/zett/politik/2021-07/oez-atten...,"Da ist Can, vielleicht drei oder vier Jahre al...",Gegen die Stille,Vor fünf Jahren starb Can Leyla bei einem Ansc...,"München, Utøya, Hanau, Rassismus, Kriminalität...",zeit,de,0,862,1,0,0,0,"['PER: Can', 'PER: Can', 'PER: Can', 'PER: Has...",3,306,3_wurde_polizei_mann_menschen,"[wurde, polizei, mann, menschen, jährige, sagt...",[Drei Wochen nach dem mörderischen Messerangri...
2929,https://zeit.de/zett/politik/2021-08/shoan-vai...,"Eine geflüchtete Person, die in Deutschland an...",Er wäre der erste Geflüchtete im Parlament,Vor zehn Jahren floh der Kurde Shoan Vaisi aus...,"Bundestagswahl, Die Linke, Bundestag, Geflücht...",zeit,de,0,576,1,0,0,0,"['LOC: Deutschland', 'PER: Shoan Vaisi', 'LOC:...",1,613,1_eu_belarus_grenze_migranten,"[eu, belarus, grenze, migranten, polen, lukasc...","[Update vom Dienstag, 16.11.2021, 08.20 Uhr: D..."
2930,https://zeit.de/zett/politik/2021-09/diversita...,Auch der neue Bundestag wird die Gesellschaft ...,Diese Menschen machen den Bundestag diverser,Zum ersten Mal sitzen zwei trans Politikerinne...,"Bundestag, Bundestagswahl, Diskriminie

In [ ]:
# frequencies migration articles vs other news topics

# frequencies migration = 1
frequency_distribution_mig_1 = merged_data[merged_data['migration'] == 1]['Name'].value_counts(normalize=True).reset_index()
frequency_distribution_mig_1.columns = ['Name', 'Proportionale Häufigkeit (migration = 1)']
frequency_distribution_mig_1['Proportionale Häufigkeit (migration = 1)'] *= 100

# Zeige den kombinierten Datensatz an
print(frequency_distribution_mig_1)


                                          Name  \
0                        2_grünen_spd_mehr_cdu   
1         0_taliban_afghanistan_kabul_menschen   
2                1_eu_belarus_grenze_migranten   
3                3_wurde_polizei_mann_menschen   
4            4_stadt_sagt_greifswald_wohnungen   
5            5_merkel_kanzlerin_angela_merkels   
6          7_sea_migranten_mittelmeer_menschen   
7                    6_rio_haiti_migranten_del   
8            8_feuerwehr_polizei_flammen_feuer   
9     10_polizei_demonstranten_neonazis_pegida   
10       9_türkei_erdogan_türkische_türkischen   
11  11_großbritannien_johnson_britische_brexit   
12               12_döner_italien_sagt_solomon   
13             18_dorf_abchasien_schule_kinder   
14              15_maryam_brüder_berlin_leiche   
15     21_frauen_polizei_männer_vergewaltigung   
16             16_syrien_amnesty_daraa_bericht   
17       13_lagus_soziales_inzidenz_gesundheit   
18        14_sprache_deutschen_wörter_deutsche   


In [ ]:
# frequencies opinion vs descriptive

# frequencies opinion =1
frequency_distribution_opi_1 = merged_data[merged_data['opinion'] == 1]['Name'].value_counts(normalize=True).reset_index()
frequency_distribution_opi_1.columns = ['Name', 'Proportionale Häufigkeit (Opinion = 1)']
frequency_distribution_opi_1['Proportionale Häufigkeit (Opinion = 1)'] *= 100

# frequencies pinion = 0
frequency_distribution_desc_0 = merged_data[merged_data['opinion'] == 0]['Name'].value_counts(normalize=True).reset_index()
frequency_distribution_desc_0.columns = ['Name', 'Proportionale Häufigkeit (Opinion = 0)']
frequency_distribution_desc_0['Proportionale Häufigkeit (Opinion = 0)'] *= 100

# Führe die beiden Tabellen nebeneinander
combined_table = pd.merge(frequency_distribution_opi_1, frequency_distribution_desc_0, on='Name', how='outer')

# Fülle fehlende Werte mit 0
combined_table = combined_table.fillna(0)

# Zeige den kombinierten Datensatz an
print(combined_table)


                                          Name  \
0                        2_grünen_spd_mehr_cdu   
1         0_taliban_afghanistan_kabul_menschen   
2                1_eu_belarus_grenze_migranten   
3                3_wurde_polizei_mann_menschen   
4            5_merkel_kanzlerin_angela_merkels   
5               15_maryam_brüder_berlin_leiche   
6         14_sprache_deutschen_wörter_deutsche   
7     10_polizei_demonstranten_neonazis_pegida   
8            4_stadt_sagt_greifswald_wohnungen   
9        9_türkei_erdogan_türkische_türkischen   
10     21_frauen_polizei_männer_vergewaltigung   
11          17_familie_kinder_kita_abschiebung   
12               12_döner_italien_sagt_solomon   
13                   6_rio_haiti_migranten_del   
14             18_dorf_abchasien_schule_kinder   
15             16_syrien_amnesty_daraa_bericht   
16  11_großbritannien_johnson_britische_brexit   
17                 20_fdp_union_jamaika_grünen   
18           8_feuerwehr_polizei_flammen_feuer   


In [ ]:
# frequencies alternative news vs conventional news

# frequencies is_alt_news =1
frequency_distribution_alt_1 = merged_data[merged_data['is_alt_news'] == 1]['Name'].value_counts(normalize=True).reset_index()
frequency_distribution_alt_1.columns = ['Name', 'Proportionale Häufigkeit (alt_news = 1)']
frequency_distribution_alt_1['Proportionale Häufigkeit (alt_news = 1)'] *= 100

# frequencies pinion = 0
frequency_distribution_conv_0 = merged_data[merged_data['is_alt_news'] == 0]['Name'].value_counts(normalize=True).reset_index()
frequency_distribution_conv_0.columns = ['Name', 'Proportionale Häufigkeit (alt_news = 0)']
frequency_distribution_conv_0['Proportionale Häufigkeit (alt_news = 0)'] *= 100

# Führe die beiden Tabellen nebeneinander
combined_table = pd.merge(frequency_distribution_alt_1, frequency_distribution_conv_0, on='Name', how='outer')

# Fülle fehlende Werte mit 0
combined_table = combined_table.fillna(0)

# Zeige den kombinierten Datensatz an
print(combined_table)


                                          Name  \
0                1_eu_belarus_grenze_migranten   
1                        2_grünen_spd_mehr_cdu   
2         0_taliban_afghanistan_kabul_menschen   
3                3_wurde_polizei_mann_menschen   
4      21_frauen_polizei_männer_vergewaltigung   
5     10_polizei_demonstranten_neonazis_pegida   
6            4_stadt_sagt_greifswald_wohnungen   
7            5_merkel_kanzlerin_angela_merkels   
8          7_sea_migranten_mittelmeer_menschen   
9               15_maryam_brüder_berlin_leiche   
10  11_großbritannien_johnson_britische_brexit   
11                   6_rio_haiti_migranten_del   
12       9_türkei_erdogan_türkische_türkischen   
13               12_döner_italien_sagt_solomon   
14             16_syrien_amnesty_daraa_bericht   
15             18_dorf_abchasien_schule_kinder   
16           8_feuerwehr_polizei_flammen_feuer   
17       13_lagus_soziales_inzidenz_gesundheit   
18        14_sprache_deutschen_wörter_deutsche   


**Code for constructing a column containing manually constructed supercategories**

In [ ]:
# Define category conversions
category_conversions = {
    'Withdrawal Afghanistan': [0],
    'Migration EU/Border Belarus': [1],
    'EU Agreement Turkey': [9],
    'Migration International': [6,11,16,19],
    'Mediterranean Sea Rescue': [7],
    'Politics': [2, 5, 20],
    'Domestic Policy Issues': [17, 18],
    'Crime': [3,8,15,21],
    'Right-wing Pressure': [4, 10],
    'Other': [12, 13, 14]
}

# Assuming you already have a DataFrame with a column named 'Category'
merged_data['Super Category'] = ''

# Assign super categories
for super_category, categories in category_conversions.items():
    merged_data.loc[merged_data['Topic'].isin(categories), 'Super Category'] = super_category

# Display the results
print(merged_data)


                                                   link  \
0     https://abendblatt.de/hamburg/kommunales/artic...   
1     https://abendblatt.de/hamburg/polizeimeldungen...   
2     https://abendblatt.de/politik/article233076529...   
3     https://abendblatt.de/politik/article233452685...   
4     https://abendblatt.de/region/schleswig-holstei...   
...                                                 ...   
2927  https://zeit.de/zeit-magazin/leben/2021-08/tan...   
2928  https://zeit.de/zett/politik/2021-07/oez-atten...   
2929  https://zeit.de/zett/politik/2021-08/shoan-vai...   
2930  https://zeit.de/zett/politik/2021-09/diversita...   
2931  https://zuerst.de/2021/08/24/immer-mehr-grenzz...   

                                                   text  \
0     Hamburg. Es gab Zeiten in der seit 2015 besteh...   
1     Cuxhaven/Hamburg. Kurz nach dem Start einer Ai...   
2     Berlin. Es sind Bilder, die einen nicht so sch...   
3     Berlin/Kiel. Daniel Günther tut, wovon andere ...

In [ ]:
merged_data.to_excel("/content/drive/MyDrive/Masterarbeit/BertClassifierMig/News/news_categorised_final_all_Mig_NER_BERTopic.xlsx", index=False)

**Experiment with random state**

In [ ]:
import numpy as np

# remove stop words (https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html)
nltk.download('stopwords')
german_stop_words = stopwords.words('german')

vectorizer_model = CountVectorizer(stop_words = german_stop_words)

# construct empty object in which results are written
ergebnisse = []

for i in range(0, 20):

    umap_model = UMAP(random_state=int(i))


    topic_model2 = BERTopic(language = "multilingual", vectorizer_model=vectorizer_model, calculate_probabilities=True, umap_model=umap_model, nr_topics = "auto")
    topics2, probs2 = topic_model2.fit_transform(list(data["text"]))

    # save the count for outliers and topics in every iteration into objects
    count_outliers = topic_model2.get_topic_info()["Count"][0]
    count_topic_0 = topic_model2.get_topic_info()["Count"][1]
    count_topics = len(topic_model2.get_topic_info()["Topic"])

    # append results of every iteration into the previously constructed empty object
    ergebnisse.append({'Iterations': i, 'Count outliers': count_outliers, 'Count Topic 0': count_topic_0,'Count topics': count_topics})

    print(i)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
ergebnisse

[{'Iterations': 0,
  'Count outliers': 7,
  'Count Topic 0': 2946,
  'Count topics': 3},
 {'Iterations': 1,
  'Count outliers': 7,
  'Count Topic 0': 2925,
  'Count topics': 4},
 {'Iterations': 2,
  'Count outliers': 7,
  'Count Topic 0': 2946,
  'Count topics': 3},
 {'Iterations': 3,
  'Count outliers': 1115,
  'Count Topic 0': 872,
  'Count topics': 18},
 {'Iterations': 4,
  'Count outliers': 7,
  'Count Topic 0': 2925,
  'Count topics': 4},
 {'Iterations': 5,
  'Count outliers': 13,
  'Count Topic 0': 2919,
  'Count topics': 4},
 {'Iterations': 6,
  'Count outliers': 1163,
  'Count Topic 0': 1044,
  'Count topics': 22},
 {'Iterations': 7,
  'Count outliers': 1093,
  'Count Topic 0': 1739,
  'Count topics': 8},
 {'Iterations': 8,
  'Count outliers': 1137,
  'Count Topic 0': 1036,
  'Count topics': 20},
 {'Iterations': 9,
  'Count outliers': 21,
  'Count Topic 0': 2900,
  'Count topics': 5},
 {'Iterations': 10,
  'Count outliers': 7,
  'Count Topic 0': 2925,
  'Count topics': 4},
 {'I

**Proportion Topics**

In [ ]:
data = pd.read_excel("/content/drive/MyDrive/Masterarbeit/BertClassifierEnv/News/news_categorised_final_all_NER_BERTopic.xlsx")
data

,link,text,title,lead,tags,source,language,is_alt_news,word_count,migration,tags_migration,identical,opinion,Entities,Topic,Count,Name,Representation,Representative_Docs
0,https://abendblatt.de/hamburg/kommunales/artic...,Hamburg. Es gab Zeiten in der seit 2015 besteh...,Rot-grüne Gereiztheiten und Sticheleien im Rat...,Die Sozialdemokraten werfen dem Koalitionspart...,Hamburger Senat,abendblatt,de,0,1082,1,0,0,0,"['LOC: Hamburg', 'ORG: SPD', 'ORG: Grünen', 'P...",1,519,1_grünen_spd_cdu_laschet,"['grünen', 'spd', 'cdu', 'laschet', 'partei', ...",['Der Regen hat endlich etwas nachgelassen an ...
1,https://abendblatt.de/hamburg/polizeimeldungen...,Cuxhaven/Hamburg. Kurz nach dem Start einer Ai...,Seenotretter holen krankes Kleinkind von „Aida...,Kurz nach dem Start der Kreuzfahrt in Hamburg ...,Kreuzfahrt,abendblatt,de,0,120,1,0,0,0,"['LOC: Cuxhaven', 'LOC: Hamburg', 'LOC: Hambur...",5,58,5_migranten_menschen_mittelmeer_sea,"['migranten', 'menschen', 'mittelmeer', 'sea',...",['Das Flüchtlingsrettungschiff Ocean Viking is...
2,https://abendblatt.de/politik/article233076529...,"Berlin. Es sind Bilder, die einen nicht so sch...","""Menschliche Überreste"" an Flugzeug gefunden –...",Verzweifelte klammerten sich in Kabul an ein U...,Afghanistan,abendblatt,de,0,371,1,0,0,0,"['LOC: Berlin', 'LOC: Kabul', 'ORG: Taliban', ...",0,1387,0_afghanistan_taliban_menschen_eu,"['afghanistan', 'taliban', 'menschen', 'eu', '...","['Update vom 15. November, 6.56 Uhr: Tausende ..."
3,https://abendblatt.de/politik/article233452685...,"Berlin/Kiel. Daniel Günther tut, wovon andere ...",Günther zu Jamaika: „Jede Partei muss Kröten s...,CDU-Ministerpräsident Daniel Günther über sein...,Interview,abendblatt,de,0,544,1,0,0,0,"['LOC: Berlin', 'LOC: Kiel', 'PER: Daniel Günt...",1,519,1_grünen_spd_cdu_laschet,"['grünen', 'spd', 'cdu', 'laschet', 'partei', ...",['Der Regen hat endlich etwas nachgelassen an ...
4,https://abendblatt.de/region/schleswig-holstei...,Helgoland. Ein Unfall auf dem Kreuzfahrtschiff...,Unfall auf der „Europa“ – Frau vor Helgoland i...,Seenotretter übernehmen auf der Nordsee eine v...,Kreuzfahrt,abendblatt,de,0,196,1,0,0,0,"['LOC: Helgoland', 'MISC: Europa', 'LOC: Helgo...",5,58,5_migranten_menschen_mittelmeer_sea,"['migranten', 'menschen', 'mittelmeer', 'sea',...",['Das Flüchtlingsrettungschiff Ocean Viking is...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2961,https://zeit.de/zeit-magazin/leben/2021-08/tan...,"Empathie verbindet Menschen, sie ermöglicht un...","""Empathie ist nicht zu verwechseln mit Mitgefühl""","Krieg, Flut, Brände: Aus der Ferne empfinden w...",NaN,zeit,de,0,104,1,0,0,0,"['LOC: Afghanistan', 'LOC: Haiti', 'PER: Tania...",0,1387,0_afghanistan_taliban_menschen_eu,"['afghanistan', 'taliban', 'menschen', 'eu', '...","['Update vom 15. November, 6.56 Uhr: Tausende ..."
2962,https://zeit.de/zett/politik/2021-07/oez-atten...,"Da ist Can, vielleicht drei oder vier Jahre al...",Gegen die Stille,Vor fünf Jahren starb Can Leyla bei einem Ansc...,"München, Utøya, Hanau, Rassismus, Kriminalität...",zeit,de,0,862,1,0,0,0,"['PER: Can', 'PER: Can', 'PER: Can', 'PER: Has...",2,477,2_wurde_sagt_polizei_mann,"['wurde', 'sagt', 'polizei', 'mann', 'deutschl...","['Es ist die Verzweiflung, die Samim J. und se..."
2963,https://zeit.de/zett/politik/2021-08/shoan-vai...,"Eine geflüchtete Person, die in Deutschland an...",Er wäre der erste Geflüchtete im Parlament,Vor zehn Jahren floh der Kurde Shoan Vaisi aus...,"Bundestagswahl, Die Linke, Bundestag, Geflücht...",zeit,de,0,576,1,0,0,0,"['LOC: Deutschland', 'PER: Shoan Vaisi', 'LOC:...",0,1387,0_afghanistan_taliban_menschen_eu,"['afghanistan', 'taliban', 'menschen', 'eu', '...","['Update vom 15. November, 6.56 Uhr: Tausende ..."
2964,https://zeit.de/zett/politik/2021-09/diversita...,Auch der neue Bundestag wird die Gesellschaft ...,Diese Menschen machen den Bundestag diverser,Zum ersten Mal sitzen zwei trans Politikerinne...,"Bundestag, Bundestagswahl, Dis

In [ ]:
# Gruppierung nach 'Topic' und 'is_alt_news' und Zählung der Kategorien
grouped = data.groupby(['Topic', 'is_alt_news']).size().unstack(fill_value=0)

grouped

is_alt_news,0,1
Topic,,
0,1224,163
1,484,35
2,461,16
3,103,3
4,55,3
5,55,3
6,43,2
7,37,0
8,30,2


In [ ]:
# Gruppierung nach 'Topic' und 'is_alt_news' und Zählung der Kategorien
grouped = data.groupby(['Name', 'is_alt_news']).size().unstack(fill_value=0)

# Berechnung des relativen Anteils pro Topic innerhalb jeder is_alt_news-Gruppe
grouped_relative = grouped.div(grouped.sum(axis=0), axis=1) * 100

print(grouped_relative)

is_alt_news                                         0          1
Name                                                            
0_afghanistan_taliban_menschen_eu           45.099484  64.682540
10_großbritannien_johnson_britische_brexit   0.957996   0.793651
11_polizei_demonstranten_neonazis_afd        0.810612   1.587302
12_frauen_männer_mädchen_leonie              0.700074   3.968254
13_lagus_soziales_inzidenz_gesundheit        0.847458   0.000000
14_lukaschenko_eu_belarus_merkel             0.589536   0.000000
15_maryam_brüder_berlin_koffer               0.515844   0.793651
16_syrien_amnesty_daraa_assad                0.478998   0.000000
17_familie_abschiebung_ovg_kinder            0.515844   0.000000
18_russland_rt_putin_depardieu               0.294768   1.984127
19_usa_einreisen_einreise_us                 0.478998   0.000000
1_grünen_spd_cdu_laschet                    17.833456  13.888889
20_klima_co2_menschen_huq                    0.442152   0.396825
21_is_11_september_us    